<a href="https://colab.research.google.com/github/anindita/dances/blob/main/analyse/optical_flow_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
YOUTUBE_ID = 'kJ3Tbmwrigk'

!pip install -q youtube-dl
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
!ffmpeg -y -loglevel info -i youtube.mp4 -ss 00:00:15 -t 2 video.mp4

In [ ]:
from scipy.io import loadmat, savemat
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
from google.colab.patches import cv2_imshow

model_1 = loadmat(f'/content/w32_{dataset}_{scale}_best.mat')
model_2 = loadmat(f'/content/w32_{dataset}_{scale}_final.mat')
model_3 = loadmat(f'/content/w48_{dataset}_{scale}_best.mat')
model_4 = loadmat(f'/content/w48_{dataset}_{scale}_final.mat')

num_imgs = len(model_1['preds'])
num_joints = len(model_1['preds'][0])

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (10,10),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 0.03))

def get_keypoints(frame_num):
  keypoints = []
  for i in range(num_joints):
    result = list(zip(model_1['preds'][frame_num][i], model_2['preds'][frame_num][i], model_3['preds'][frame_num][i], model_4['preds'][frame_num][i]))
    x = int(sum(list(result[0])) / len(list(result[0])))
    y = int(sum(list(result[1])) / len(list(result[1])))
    keypoints.append([[x,y]])
  return np.array(keypoints, dtype=np.float32)

def annot_img(img, keypoints):
  for i, ps in enumerate(keypoints):
    x,y = ps.ravel()
    cv2.circle(img, (x, y), 3, colors[i], -1)

# [
#  [[208. 758.]], 
#  [[208. 770.]], 
#  [[443. 473.]], 
#  [[230. 757.]]
# ]

def dist(x1,y1,x2,y2):
  return math.hypot(x2 - x1, y2 - y1)

def analyse_points(prev_ps, opt_ps, opt_st, curr_ps):
  keypoints = []
  for i, ps in enumerate(curr_ps):
    x, y = ps.ravel()

    if opt_st[i][0] == 0:
      keypoints.append([[x,y]])
      continue

    prev_x, prev_y = prev_ps[i].ravel()
    opt_x, opt_y = opt_ps[i].ravel()

    curr_dist = dist(prev_x, prev_y, x, y)
    opt_dist = dist(prev_x, prev_y, opt_x, opt_y)

    if curr_dist - opt_dist < 20:
      keypoints.append([[x,y]])
    else:
      keypoints.append([[opt_x, opt_y]])

    #keypoints.append([[x,y]])

  return np.array(keypoints, dtype=np.float32)


In [ ]:
p0 = get_keypoints(0)
kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
out = cv2.VideoWriter('output_video.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, video_size)

for i in range(1, num_imgs):

  old_img = cv2.imread(f'/content/deep-high-resolution-net.pytorch/data/mpii/images/frame{i-1}.jpg')
  img = cv2.imread(f'/content/deep-high-resolution-net.pytorch/data/mpii/images/frame{i}.jpg')

  old_gray = cv2.cvtColor(old_img, cv2.COLOR_BGR2GRAY)
  old_gray = cv2.filter2D(old_gray, -1, kernel)
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_gray = cv2.filter2D(img_gray, -1, kernel)

  p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, img_gray, p0, None, **lk_params)

  curr_points = get_keypoints(i)
  new_keypoints = analyse_points(p0, p1, st, curr_points)
  annot_img(img, new_keypoints)

  p0 = new_keypoints.copy()
  
  cv2_imshow(img)
  out.write(img)
  
cv2.destroyAllWindows()
out.release()

extra

In [ ]:
from scipy.io import loadmat, savemat
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
from google.colab.patches import cv2_imshow

num_imgs = len(smooths[0][0])
num_joints = len(smooths)

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (10,10),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 0.03))

def get_keypoints(frame_num):
  keypoints = []
  for i in range(num_joints):
    x = results[i][0][frame_num]
    y = results[i][1][frame_num]
    keypoints.append([[x,y]])
  return np.array(keypoints, dtype=np.float32)

def annot_img(img, keypoints):
  for i, ps in enumerate(keypoints):
    x,y = ps.ravel()
    cv2.circle(img, (x, y), 3, colors[i], -1)

# [
#  [[208. 758.]], 
#  [[208. 770.]], 
#  [[443. 473.]], 
#  [[230. 757.]]
# ]

def dist(x1,y1,x2,y2):
  return math.hypot(x2 - x1, y2 - y1)

def analyse_points(prev_ps, opt_ps, opt_st, curr_ps):
  keypoints = []
  for i, ps in enumerate(curr_ps):
    x, y = ps.ravel()

    if opt_st[i][0] == 0:
      keypoints.append([[x,y]])
      continue

    prev_x, prev_y = prev_ps[i].ravel()
    opt_x, opt_y = opt_ps[i].ravel()

    curr_dist = dist(prev_x, prev_y, x, y)
    opt_dist = dist(prev_x, prev_y, opt_x, opt_y)

    if curr_dist - opt_dist < 30:
      keypoints.append([[x,y]])
    else:
      keypoints.append([[opt_x, opt_y]])

    #keypoints.append([[x,y]])

  return np.array(keypoints, dtype=np.float32)


p0 = get_keypoints(0)
kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
out = cv2.VideoWriter('optical_flow.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, video_size)

for i in range(1, num_imgs):

  old_img = cv2.imread(f'/content/deep-high-resolution-net.pytorch/data/mpii/images/frame{i-1}.jpg')
  img = cv2.imread(f'/content/deep-high-resolution-net.pytorch/data/mpii/images/frame{i}.jpg')

  old_gray = cv2.cvtColor(old_img, cv2.COLOR_BGR2GRAY)
  old_gray = cv2.filter2D(old_gray, -1, kernel)
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_gray = cv2.filter2D(img_gray, -1, kernel)

  p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, img_gray, p0, None, **lk_params)

  curr_points = get_keypoints(i)
  new_keypoints = analyse_points(p0, p1, st, curr_points)
  annot_img(img, new_keypoints)

  p0 = new_keypoints.copy()
  
  cv2_imshow(img)
  out.write(img)
  
cv2.destroyAllWindows()
out.release()